In [ ]:
!pip install feast[aws] joblib s3fs awscli xgboost==0.90

     |████████████████████████████████| 1.1 MB 8.0 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
     |████████████████████████████████| 8.4 MB 35.1 MB/s 
     |████████████████████████████████| 271 kB 28.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 71.6 MB/s 
     |████████████████████████████████| 132 kB 60.5 MB/s 
     |████████████████████████████████| 132 kB 65.8 MB/s 
     |████████████████████████████████| 132 kB 53.1 MB/s 
     |████████████████████████████████| 132 kB 57.3 MB/s 
     |████████████████████████████████| 132 kB 57.2 MB/s 
     |████████████████████████████████| 132 kB 39.0 MB/s 
     |████████████████████████████████| 132 kB 51.2 MB/s 
     |████████████████████████████████| 132 kB 38.4 MB/s 
     |████████████████████████████████| 132 kB 57.8 MB/s 
     |████████████████████████████████| 132 kB 56.4 MB/s 
     |████████████████████████████████| 132 kB 50.6 MB/s 
     |█████████████

In [ ]:
## if you have feature repo which can be clone from Airflow environment use this. 
## (if it is private repo, you might have to figure out a way to set up api key)
!git clone <feature_repo_url>

In [ ]:
## If you have stored feature repo in s3 use this cell
!aws s3 cp s3://<s3_bucket>/customer_segmentation . --recursive

In [ ]:

import boto3
import os

# os.environ["AWS_ACCESS_KEY_ID"] = "<aws_key>"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "<aws_secret>"
# os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

model_name = "customer_segment-v0.0"
s3 = boto3.client('s3')
s3.download_file("feast-demo-mar-2022", f"model-repo/{model_name}", model_name)

In [ ]:
def fetch_customers_from_raw_data():
  ## todo: code to fetch customers from raw data
  return ["12747.0", "12841.0", "12849.0", "12854.0", "12863.0"]

customer_to_be_scored = fetch_customers_from_raw_data()  

In [ ]:
import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame(data={
    "customerid": customer_to_be_scored,
    "event_timestamp": datetime.now()
})
entity_df.head()

,CustomerID,event_timestamp
0,12747.0,2022-03-27 05:00:50.309139
1,12841.0,2022-03-27 05:00:50.309139
2,12849.0,2022-03-27 05:00:50.309139
3,12854.0,2022-03-27 05:00:50.309139
4,12863.0,2022-03-27 05:00:50.309139


In [ ]:
%cd customer_segmentation

from feast import FeatureStore
store = FeatureStore(repo_path=".")

/content/customer_segmentation
Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage


/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]


In [ ]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
              "customer_rfm_features:recency", 
              "customer_rfm_features:MonetaryValue", 
              "customer_rfm_features:R", 
              "customer_rfm_features:M",
              "customer_rfm_features:RFMScore",
              "customer_rfm_features:SegmentHighValue",
              "customer_rfm_features:SegmentLowValue",
              "customer_rfm_features:SegmentMidValue"
          ]
    )
feature_data = job.to_df()
feature_data = feature_data.dropna()
feature_data.head()

/usr/local/lib/python3.7/dist-packages/feast/infra/offline_stores/redshift_source.py:55: DeprecationWarning: Starting in Feast 0.21, Feast will require either a name for a data source (if using query) or `table`.
  DeprecationWarning,


,customerid,event_timestamp,recency,monetaryvalue,r,m,rfmscore,segmenthighvalue,segmentlowvalue,segmentmidvalue
0,12747.0,2022-03-27 05:00:50.309139,7,1082.09,3,3,8,1,0,0
1,12863.0,2022-03-27 05:00:50.309139,7,-17.00,3,0,3,0,1,0
2,12854.0,2022-03-27 05:00:50.309139,50,401.82,1,2,5,0,0,1
3,12849.0,2022-03-27 05:00:50.309139,77,704.50,0,2,4,0,1,0
4,12841.0,2022-03-27 05:00:50.309139,31,548.34,1,2,6,0,0,1


In [ ]:
pred_feature_data = feature_data

In [ ]:
import joblib

features = pred_feature_data.drop(['customerid', 'event_timestamp'], axis=1)
loaded_model = joblib.load('./customer_segment-v0.0')
prediction = loaded_model.predict(features)
prediction.tolist()

[0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
file_name = f"customer_ltv_pred_results_{datetime.now()}.parquet"
pred_feature_data["predicted_ltvcluster"] = prediction.tolist()
s3_url = f's3://feast-demo-mar-2022/prediction_results/{file_name}'
pred_feature_data.to_parquet(s3_url)

03/27/2022 05:25:34 AM INFO:Found credentials in environment variables.
